<a href="https://colab.research.google.com/github/jobsiggang/2024AIClass/blob/main/%EC%99%84%EB%A3%8C5_%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EA%B8%B0%EC%B4%88_%EC%B1%97%EB%B4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 언어모델 활용으로 심리상담 챗봇 만들기
## 모델 - 허깅페이스 https://huggingface.co/
## 데이터 파일 -
https://drive.google.com/file/d/1mNG8Vw2_Q2hxOjsr4Jgcn5wo-68ppj-3/view?usp=sharing







In [ ]:
!pip install -U sentence-transformers

In [ ]:
# sentence_transformers 모듈에서 SentenceTransformer 클래스 임포트
from sentence_transformers import SentenceTransformer

# 문장 데이터 준비
sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]

# SentenceTransformer 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 문장 임베딩 생성
embeddings = model.encode(sentences)
print(embeddings)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 첫 번째 문장에 대한 임베딩 생성
embeddings1 = model.encode("안녕하세요 반갑습니다")
# 두 번째 문장에 대한 임베딩 생성
embeddings2 = model.encode("싫어 친구야")

# 두 벡터 간의 코사인 유사도 계산
result = cosine_similarity([embeddings1], [embeddings2])
result


In [ ]:
import pandas as pd

# pandas 라이브러리 임포트

# JSON 파일을 데이터프레임으로 읽어들임
df = pd.read_json("/content/drive/MyDrive/aidata/counsel_data.json")


In [ ]:
# 데이터프레임의 "사용자" 열에 있는 각 문장을 SentenceTransformer 모델을 사용하여 임베딩하여 새로운 "embedding" 열에 추가함
df["embedding"] = df["사용자"].map(lambda x: list(model.encode(x)))

# 데이터프레임의 처음 몇 개 행을 출력하여 확인
print(df.head())


In [ ]:
text="생각대로 안되면 화가 나요"
# 주어진 텍스트를 SentenceTransformer 모델을 사용하여 임베딩함
text_embedding = model.encode(text)

# 임베딩된 벡터 출력
print(text_embedding)


In [ ]:
# 각 문장의 임베딩과 주어진 텍스트의 임베딩 간의 코사인 유사도 계산 후 "similarity" 열에 추가함
df['similarity'] = df["embedding"].map(lambda x: cosine_similarity([text_embedding], [x]).squeeze())

# 유사도 열 출력
print(df["similarity"])


In [ ]:
# 유사도가 가장 높은 문장의 인덱스를 찾아 해당 행 선택
answer = df.loc[df["similarity"].idxmax()]

# 선택된 행에서 "챗봇" 열의 값을 출력
print(answer["챗봇"])


'화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.'

# 웹앱으로 서비스

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def chatbot(user):
    # 사용자 입력에 대한 임베딩 생성
    text_embedding = model.encode(user)

    # 각 문장과 사용자 입력 간의 코사인 유사도 계산 후 유사도 열에 추가
    df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([text_embedding], [x]).squeeze())

    # 유사도가 가장 높은 문장 선택
    answer = df.loc[df['similarity'].idxmax()]

    # 선택된 문장의 챗봇 열 값 반환
    return answer["챗봇"]

# Gradio 인터페이스 생성 및 런칭
demo = gr.Interface(fn=chatbot, inputs='text', outputs='text')
demo.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5b65d68d4222385fd1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5b65d68d4222385fd1.gradio.live
